In [1]:
!pip install google-generativeai langgraph python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached langgraph-1.0.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_auth-2.41.1-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_prebuilt-1.0.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached googleapis_common_protos-1.71.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached ormsgpack-1.11.0-cp312-cp312-win

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.26 requires numpy>=1.26.2; python_version < "3.13", which is not installed.
tensorflow-intel 2.17.0 requires keras>=3.2.0, which is not installed.
tensorflow-intel 2.17.0 requires ml-dtypes<0.5.0,>=0.3.1, which is not installed.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", which is not installed.
tensorflow-intel 2.17.0 requires tensorboard<2.18,>=2.17, which is not installed.
tensorflow-intel 2.17.0 requires wrapt>=1.11.0, which is not installed.
langchain 0.3.26 requires langchain-core<1.0.0,>=0.3.66, but you have langchain-core 1.0.1 which is incompatible.
langchain-community 0.3.26 requires langchain-core<1.0.0,>=0.3.66, but you have langchain-core 1.0.1 which is incompatible.
langchain-text-splitters 0.3.8 requires langchain-core<1.0.0,>=0.3.51, but 

In [2]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import google.generativeai as genai
import os

In [3]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

In [4]:
class ChatState(TypedDict):
    messages: List[str]

In [5]:
def chatbot_node(state: ChatState) -> ChatState:
    model = genai.GenerativeModel("gemini-2.0-flash")
    user_message = state["messages"][-1]
    response = model.generate_content(user_message)
    return {"messages": state["messages"] + [response.text]}


In [6]:
graph = StateGraph(ChatState)
graph.add_node("chatbot", chatbot_node)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

app = graph.compile()

In [7]:
final_state = app.invoke({"messages": ["What do you know about LangGraph?"]})
print("Chatbot:", final_state["messages"][-1])

Chatbot: LangGraph is a framework built on top of LangChain that's designed to make it easier to build **robust and stateful multi-actor applications**, especially those driven by LLMs (Large Language Models). Think of it as a way to orchestrate complex conversations and interactions between different agents or steps in a sophisticated workflow.

Here's a breakdown of what I know about LangGraph:

**Key Concepts:**

* **Graphs:**  The core idea is representing the application's logic as a directed graph.  Nodes in the graph represent individual states or "actors" (like tools, agents, or even simple data transformations), and edges represent the flow of information between them.  This graph structure makes the logic more visual, manageable, and scalable.
* **Nodes:** These are the fundamental building blocks. Each node performs a specific task. They can be:
    * **Models (LLMs):**  This is probably the most common, using an LLM to generate text, answer questions, etc.
    * **Tools:** 